In [ ]:
pip install qiskit opencv-python pycryptodome

In [ ]:
import cv2
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.providers.aer import AerSimulator
from Crypto.Cipher import AES
import base64

# 1. Generate Quantum Key (BB84 Simulation)
def generate_quantum_key():
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.measure(0, 0)
    simulator = AerSimulator()
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    counts = result.get_counts()
    key_bit = max(counts, key=counts.get)
    quantum_key = (key_bit * 16)[:16]  # 16-bit AES-compatible
    return quantum_key

# 2. Embed Quantum Key in Image (LSB Steganography)
def embed_key_in_image(image_path, key, output_path="encrypted_image.png"):
    img = cv2.imread(image_path)
    binary_key = ''.join(format(ord(c), '08b') for c in key)
    index = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(3):  # R, G, B
                if index < len(binary_key):
                    img[i, j, k] = (img[i, j, k] & ~1) | int(binary_key[index])
                    index += 1
    cv2.imwrite(output_path, img)

# 3. Extract Key from Image
def extract_key_from_image(image_path, key_length=16):
    img = cv2.imread(image_path)
    binary_key = ''
    index = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(3):
                if index < key_length * 8:
                    binary_key += str(img[i, j, k] & 1)
                    index += 1
    chars = [binary_key[i:i+8] for i in range(0, len(binary_key), 8)]
    return ''.join(chr(int(c, 2)) for c in chars)

# 4. AES-256 Encryption and Decryption
def pad_key(key):
    return key.ljust(32)[:32]  # pad or truncate to 32 bytes for AES-256

def encrypt_message(message, key):
    cipher = AES.new(pad_key(key).encode(), AES.MODE_ECB)
    encrypted = cipher.encrypt(message.ljust(32).encode())
    return base64.b64encode(encrypted).decode()

def decrypt_message(encrypted_message, key):
    cipher = AES.new(pad_key(key).encode(), AES.MODE_ECB)
    decrypted = cipher.decrypt(base64.b64decode(encrypted_message))
    return decrypted.strip().decode()

# === Run the Flow ===
image_path = "sample.png"  # Use any .png image with enough size
message = "Confidential: QKD Project"

# Step 1: Generate Quantum Key
q_key = generate_quantum_key()
print("Quantum Key:", q_key)

# Step 2: Embed into image
embed_key_in_image(image_path, q_key)

# Step 3: Extract the key
extracted_key = extract_key_from_image("encrypted_image.png")
print("Extracted Key:", extracted_key)

# Step 4: Encrypt and Decrypt
encrypted_msg = encrypt_message(message, extracted_key)
print("Encrypted:", encrypted_msg)

decrypted_msg = decrypt_message(encrypted_msg, extracted_key)
print("Decrypted:", decrypted_msg)